In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [6]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello,how,are, you?")

['Hello', 'how', 'are', 'you?']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# prompt = template.format_messages(
#     max_items=10,
#     question="What are the colors?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

In [14]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international Poet. You create poetry about programming languages. you use only one programming langauage to one poetry"),
    ("human", "I want to make {language} poetry."),
])

poet_chain = poet_prompt | chat

poetry_commentator_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a poetry commentator specialized on explaining programming language's poetry."),
    ("human", "{poetry}")
])

poetry_commentator_chain = poetry_commentator_prompt | chat

final_chain = {"poetry": poet_chain} | poetry_commentator_chain

final_chain.invoke({
    "language": "python"
})

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question" : "What do you know about France?",
        "answer" : """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    }
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "movie": "Mad Max: Fury Road",
        "answer": """
        Main Cast:
        Tom Hardy as Max Rockatansky
	    Charlize Theron as Imperator Furiosa
	    Nicholas Hoult as Nux
	    Budget: Approximately $150-185 million
	    Box Office Gross: Approximately $375.6 million
	    Genre: Post-Apocalyptic, Action
	    Synopsis: “Mad Max: Fury Road” is set in a post-apocalyptic world where Max, haunted by his past, becomes involved in a struggle led by Furiosa to escape from the tyrannical Immortan Joe. Together, they embark on a high-octane chase across the desert, fighting for survival and freedom.
        """,
    },
    {
        "movie": "Titanic",
        "answer": """
        I know this:
        Director: James Cameron
	    Main Cast:
		Leonardo DiCaprio as Jack Dawson
		Kate Winslet as Rose DeWitt Bukater
		Billy Zane as Cal Hockley
		Budget: Approximately $200 million
		Box Office Gross: Approximately $2.202 billion
		Genre: Romance, Drama, Disaster
		Synopsis:“Titanic” is a tragic love story set against the backdrop of the RMS Titanic’s ill-fated maiden voyage in 1912. It follows the romance between Jack, a poor artist, and Rose, a wealthy woman trapped in an unhappy engagement. Their love faces immense challenges as the ship meets its catastrophic fate.
        """,
    },
    {
        "movie": "Avatar",
        "answer": """
        I know this:
        Director: James Cameron
		Main Cast:
		Sam Worthington as Jake Sully
		Zoe Saldana as Neytiri
		Sigourney Weaver as Grace Augustine
		Budget: Approximately $237 million
		Box Office Gross: Approximately $2.923 billion
		Genre: Sci-Fi, Action, Adventure
		Synopsis:“Avatar” is set on the lush alien world of Pandora in the 22nd century. The story follows Jake Sully, a paralyzed ex-Marine who participates in the Avatar Program. Through his avatar, Jake immerses himself in the culture of the Na’vi people and eventually leads them in a battle to protect their world from human exploitation.
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie": "Parasite"})



        I know this:
        Director: Bong Joon-ho
		Main Cast:
		Kang-ho Song as Kim Ki-taek
		Sun-kyun Lee as Park Dong-ik
		Yeo-jeong Jo as Choi Yeon-gyo
		Budget: Approximately $11 million
		Box Office Gross: Approximately $266.8 million
		Genre: Thriller, Drama, Dark Comedy
		Synopsis:“Parasite” is a South Korean film that revolves around the Kim family, who cunningly infiltrate the wealthy Park family's household by posing as unrelated skilled workers. As their lives become increasingly intertwined, dark secrets and unexpected events unfold, leading to a gripping tale of class struggle and deception.

AIMessageChunk(content="\n        I know this:\n        Director: Bong Joon-ho\n\t\tMain Cast:\n\t\tKang-ho Song as Kim Ki-taek\n\t\tSun-kyun Lee as Park Dong-ik\n\t\tYeo-jeong Jo as Choi Yeon-gyo\n\t\tBudget: Approximately $11 million\n\t\tBox Office Gross: Approximately $266.8 million\n\t\tGenre: Thriller, Drama, Dark Comedy\n\t\tSynopsis:“Parasite” is a South Korean film that revolves around the Kim family, who cunningly infiltrate the wealthy Park family's household by posing as unrelated skilled workers. As their lives become increasingly intertwined, dark secrets and unexpected events unfold, leading to a gripping tale of class struggle and deception.")